In [ ]:
import pandas as pd
import numpy as np
import pathlib
import itertools

In [ ]:
data_folder = pathlib.Path('./data')
file_list = list(data_folder.glob('*.xlsx'))
question_labels = ['Q1', 'Q2', 'Q3','Q4','Q5']
column_names = ['name'] + question_labels 

# read first file to grab student names
file_path = file_list[0]
df = pd.read_excel(file_path)
df = df.drop(0)
df.columns = column_names
 
student_names = list(df['name'])
headr = pd.MultiIndex.from_tuples(itertools.product(student_names,question_labels))
# construct the combined scores dataframe
cols = pd.MultiIndex.from_tuples(headr)
df_combined = pd.DataFrame([], student_names, cols)
# df

In [ ]:
# remembering to drop scores people give to themselves.
# @WARNING: df_combined row indices won't be accurate if file and row sorting don't match, 
#          however finals results will still be accurate
for s_name,file_path in zip(student_names, file_list):
    print('Reading file from: {}'.format(s_name))
    df = pd.read_excel(file_path)
    # drop first row, its just explanations
    df = df.drop(0)
    # some students don't have all the columns for some reason.
    if len(df.columns) == 5:
        df["Q5"] = np.nan
    # print(len(df.columns))    
    df.columns = column_names
    df = df.set_index('name')
    df_combined.loc[s_name] = df.values.flatten()
    
# df_combined

In [ ]:
# calculate scores now
# count the number of valid entries for each question
valid_count = df_combined.count()
# tally up all the scores for questions
sums_question = df_combined.sum()
# normalize the scores
normalized_question_scores = sums_question / valid_count
# sum question scores into a single score (out of 5)
scores = normalized_question_scores.sum(level=[0])
# scores out of 100
scores = scores * 20
# print the class average
print('Class average is: {} out of 100'.format(scores.mean()))

In [ ]:
# Export the scores out to a CSV.
scores.to_csv(data_folder / '../presentation_scores.csv')